In [1]:
!pip install pandas scikit-learn optuna seaborn

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import optuna
import os
from include.utils import incrementar_dados_aleatorios_csv, categorizar_imc
from include.utils import prever_encargos_futuros, segmentacao_de_risco, analise_de_sensibilidade, otimizacao_de_recursos, planejamento_estrategico

In [3]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

**1. Engenharia de Recursos**

Novos recursos derivados dos dados existentes

In [4]:
# Obtém a largura do terminal - uso mais embaixo nos prints
terminal_width = os.get_terminal_size().columns

# leia os dados CSV indicados no site da FIAP
dados = pd.read_csv('../base_dados.csv', encoding='latin-1')

# aqui chamo a função incrementar_dados_aleatorios, para aumentar o volume da variável dados
dados = incrementar_dados_aleatorios_csv(dados)

# Aplicar a função para categorizar IMC e criar uma nova coluna 'Categoria IMC'
dados['Categoria IMC'] = dados['IMC'].apply(categorizar_imc)

# Mapeamento das regiões para valores numéricos
regiao_numerica = {'sudoeste': 1, 'sudeste': 2, 'nordeste': 3, 'noroeste': 4}

# Convertendo os valores da coluna 'Região' para valores numéricos usando o mapeamento
dados['Região_numerica'] = dados['Região'].map(regiao_numerica)

**2. Limpeza de Dados**

In [5]:
# Remover linhas com valores NaN
dados = dados.dropna()

# Substituir valores nulos por zero
dados['Filhos'] = dados['Filhos'].fillna(0)
dados['Filhos'] = dados['Filhos'].astype(int)
dados['Idade'] = dados['Idade'].fillna(0)
dados['Idade'] = dados['Idade'].astype(int)
dados['Encargos'] = dados['Encargos'].fillna(0)

dados['Gênero'] = dados['Gênero'].str.replace('nan', 'Não informado')

**3. Pré-processamento**

In [6]:
# Dividir os dados em features (X) e target (y)
X = dados.drop('Encargos', axis=1)  # Coluna alvo
y = dados['Encargos']

# Codificar variáveis categóricas usando one-hot encoding
encoder = OneHotEncoder(drop='first')
X_encoded = encoder.fit_transform(X[['Gênero', 'Região']])

# Concatenar features codificadas com as restantes
X_encoded = pd.DataFrame(X_encoded.toarray(), columns=encoder.get_feature_names_out(['Gênero', 'Região']), index=X.index)
X = pd.concat([X.drop(['Gênero', 'Região'], axis=1), X_encoded], axis=1)

# Codificar variável categórica 'IMC' com LabelEncoder
label_encoder = LabelEncoder()
X['Categoria IMC'] = label_encoder.fit_transform(X['Categoria IMC'])

# Substituir valores 'sim' e 'não' por 1 e 0
X['Fumante'] = X['Fumante'].map({'sim': 1, 'não': 0})

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

correlation_matrix = X_train.corr()
print('Visualizar a matriz de correlação:')
print(correlation_matrix)

Visualizar a matriz de correlação:
                     Idade       IMC        Filhos   Fumante  Categoria IMC  \
Idade             1.000000 -0.118711 -1.806146e-02 -0.217938       0.060241   
IMC              -0.118711  1.000000 -1.768686e-02  0.192693      -0.296942   
Filhos           -0.018061 -0.017687  1.000000e+00  0.055706       0.044948   
Fumante          -0.217938  0.192693  5.570597e-02  1.000000       0.009032   
Categoria IMC     0.060241 -0.296942  4.494821e-02  0.009032       1.000000   
Região_numerica   0.038374 -0.145003  3.664541e-17 -0.214055       0.059858   
Gênero_masculino  0.036435  0.005248  2.778222e-02 -0.134176       0.007983   
Região_noroeste   0.009498 -0.141363  4.088087e-02 -0.160139       0.086590   
Região_sudeste    0.097834 -0.037978 -2.319207e-02  0.023833       0.141801   
Região_sudoeste  -0.088912  0.135608  3.088418e-02  0.189169      -0.097744   

                  Região_numerica  Gênero_masculino  Região_noroeste  \
Idade                3.

**4. Treinamento do Modelo**

In [8]:
# Adicionando mais modelos ao dicionário de modelos
models = {
    'Random Forest Regressor': RandomForestRegressor(),
    'Linear Regression': LinearRegression(),
    'Support Vector Regression': SVR(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'K-Nearest Neighbors Regressor': KNeighborsRegressor(),
    'Gradient Boosting Regressor': GradientBoostingRegressor()
}

# Treinamento e avaliação dos modelos
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    # Imprime uma linha separadora com base na largura do terminal
    print('-' * terminal_width)
    print(f'{name} MSE: {round(mse,2)}')
    # Calcular acurácia média usando validação cruzada
    scores = cross_val_score(model, X_train, y_train, cv=5)
    print(f'Acurácia média com validação cruzada para {name}: {round(scores.mean(),2)}')
    
    # Calcula e exibe a importância das características para o modelo atual
    if hasattr(model, 'feature_importances_'):
        importance_df = pd.DataFrame({'Colunas': X_train.columns, 'Importância': model.feature_importances_})
        importance_df = importance_df.sort_values(by='Importância', ascending=False)
        print(f"Importância das características para {name}:")
        print(round(importance_df,2))
    elif isinstance(model, LinearRegression):
        importance_df = pd.DataFrame({'Colunas': X_train.columns, 'Coeficiente': model.coef_})
        importance_df = importance_df.sort_values(by='Coeficiente', ascending=False)
        print(f"Coeficientes das características para {name}:")
        print(round(importance_df,2))
    elif isinstance(model, SVR):
        support_indices = model.support_
        support_features = X_train.iloc[support_indices]
        feature_importance_svr = support_features.mean(axis=0)
        importance_df = pd.DataFrame({'Colunas': X_train.columns, 'Importância': feature_importance_svr})
        importance_df = importance_df.sort_values(by='Importância', ascending=False)
        print(f"Importância das características para {name}:")
        print(round(importance_df,2))
    else:
        print(f"Não é possível calcular a importância das características para o modelo {name}")

--------------------------------------------------------------------------------------------------------------------------------------------------------------
Random Forest Regressor MSE: 167653078.24
Acurácia média com validação cruzada para Random Forest Regressor: 0.29
Importância das características para Random Forest Regressor:
            Colunas  Importância
0             Idade         0.45
1               IMC         0.35
2            Filhos         0.06
5   Região_numerica         0.04
3           Fumante         0.03
4     Categoria IMC         0.02
9   Região_sudoeste         0.02
6  Gênero_masculino         0.01
7   Região_noroeste         0.01
8    Região_sudeste         0.01
--------------------------------------------------------------------------------------------------------------------------------------------------------------
Linear Regression MSE: 185683950.57
Acurácia média com validação cruzada para Linear Regression: 0.46
Coeficientes das características para Lin

**5. Comparação de Modelos**

In [9]:
# Seleção de modelo com base na menor pontuação de validação cruzada
best_model_name = min(models, key=lambda model: cross_val_score(models[model], X_train, y_train, cv=5).mean())

**6. Seleção de Modelo**

In [10]:
best_model = models[best_model_name]
print(f'Best Model: {best_model_name}')

Best Model: Decision Tree Regressor


**7. Otimização de Modelo**

In [11]:
# Otimização de hiperparâmetros com Optuna (exemplo com RandomForestRegressor)
def objective(trial):
    if best_model_name == 'Random Forest Regressor':
        n_estimators = trial.suggest_int('n_estimators', 10, 100)
        max_depth = trial.suggest_int('max_depth', 2, 32)
        model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
    elif best_model_name == 'Decision Tree Regressor':
        max_depth = trial.suggest_int('max_depth', 2, 32)
        model = DecisionTreeRegressor(max_depth=max_depth)
    else:
        model = models[best_model_name]  # Use the default hyperparameters for other models
    score = cross_val_score(model, X_train, y_train, n_jobs=-1, cv=3, scoring='neg_mean_squared_error').mean()
    return score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2024-04-08 18:17:09,051] A new study created in memory with name: no-name-1b10bb97-48a0-428b-b7cc-56c7eac420e3
[I 2024-04-08 18:17:11,160] Trial 0 finished with value: -793100528.3033161 and parameters: {'max_depth': 13}. Best is trial 0 with value: -793100528.3033161.
[I 2024-04-08 18:17:12,721] Trial 1 finished with value: -625262832.0530726 and parameters: {'max_depth': 4}. Best is trial 0 with value: -793100528.3033161.
[I 2024-04-08 18:17:14,298] Trial 2 finished with value: -759037205.7439976 and parameters: {'max_depth': 30}. Best is trial 0 with value: -793100528.3033161.
[I 2024-04-08 18:17:14,328] Trial 3 finished with value: -809584168.8738083 and parameters: {'max_depth': 26}. Best is trial 3 with value: -809584168.8738083.
[I 2024-04-08 18:17:14,355] Trial 4 finished with value: -787361524.2672821 and parameters: {'max_depth': 16}. Best is trial 3 with value: -809584168.8738083.
[I 2024-04-08 18:17:14,371] Trial 5 finished with value: -757427724.5940858 and parameters: 

In [12]:
best_params = study.best_params
best_model.set_params(**best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Optimized {best_model_name} MSE: {round(mse,2)}')

Optimized Decision Tree Regressor MSE: 301541129.85


In [13]:
# Obter a importância das características apenas para modelos que suportam esse atributo
if hasattr(best_model, 'feature_importances_'):
    # Obter a importância das características do modelo
    feature_importances = best_model.feature_importances_
    
    # Criar um DataFrame para visualizar a importância das características
    importance_df = pd.DataFrame({'Colunas': X_train.columns, 'Importância': feature_importances})

    # Ordenar as características por importância
    importance_df = importance_df.sort_values(by='Importância', ascending=False)
    
    print("\nQuais características individuais têm maior impacto nos custos médicos cobrados pelo seguro de saúde?")
    print(round(importance_df,2))

    # Selecionar as características mais importantes (por exemplo, as 10 mais importantes)
    top_features = importance_df.head(10)['Colunas'].tolist()  # Ajuste o número conforme necessário
    
    # Calcular a matriz de correlação apenas para as características mais importantes
    correlation_matrix = X_train[top_features].corr()
    
    # Visualizar a matriz de correlação
    print("\nExiste alguma correlação entre certas características (por exemplo, idade, IMC) e os custos médicos?")
    print(correlation_matrix)
else:
    print("O modelo selecionado não suporta o cálculo de importância das características.")


Quais características individuais têm maior impacto nos custos médicos cobrados pelo seguro de saúde?
            Colunas  Importância
0             Idade         0.45
1               IMC         0.34
3           Fumante         0.07
2            Filhos         0.05
9   Região_sudoeste         0.03
5   Região_numerica         0.03
6  Gênero_masculino         0.03
7   Região_noroeste         0.00
8    Região_sudeste         0.00
4     Categoria IMC         0.00

Existe alguma correlação entre certas características (por exemplo, idade, IMC) e os custos médicos?
                     Idade       IMC   Fumante        Filhos  Região_sudoeste  \
Idade             1.000000 -0.118711 -0.217938 -1.806146e-02        -0.088912   
IMC              -0.118711  1.000000  0.192693 -1.768686e-02         0.135608   
Fumante          -0.217938  0.192693  1.000000  5.570597e-02         0.189169   
Filhos           -0.018061 -0.017687  0.055706  1.000000e+00         0.030884   
Região_sudoeste  -0.088912 

In [14]:
# Definição de dados futuros - 50 linhas
dados_futuros = pd.DataFrame(np.random.rand(50, len(X_train.columns)), columns=X_train.columns)
print(dados_futuros.head())

encargos_futuros = prever_encargos_futuros(best_model, dados_futuros)
print("\nModelo treinado para fazer previsões dos encargos futuros:")
print(encargos_futuros)

grupos_risco = segmentacao_de_risco(best_model, dados_futuros)
print("\nGrupos de indivíduos com diferentes níveis de risco:")
print(grupos_risco)

custos_previstos_alterados = analise_de_sensibilidade(best_model, dados_futuros, 'Idade', novo_valor=40)
print("\nAnálises de sensibilidade para entender o impacto das mudanças nas características dos segurados nos encargos previstos:")
print(custos_previstos_alterados)

recursos_otimizados = otimizacao_de_recursos(encargos_futuros)
print("\nUtilizando as previsões dos encargos para otimizar a alocação de recursos:")
print(recursos_otimizados)

planos_estrategicos = planejamento_estrategico(best_model, dados_futuros)
print("\nUtilizando as informações obtidas com o modelo para desenvolver planos estratégicos:")
print(planos_estrategicos)

      Idade       IMC    Filhos   Fumante  Categoria IMC  Região_numerica  \
0  0.950703  0.577384  0.538600  0.802920       0.878742         0.034599   
1  0.767651  0.717420  0.805408  0.145118       0.577323         0.501070   
2  0.463857  0.731918  0.598147  0.380161       0.513174         0.578792   
3  0.672585  0.414666  0.217714  0.795809       0.861901         0.143581   
4  0.914620  0.944972  0.024718  0.709574       0.451012         0.834207   

   Gênero_masculino  Região_noroeste  Região_sudeste  Região_sudoeste  
0          0.829391         0.103225        0.552824         0.716024  
1          0.043287         0.952611        0.533182         0.780849  
2          0.925900         0.479726        0.400840         0.998053  
3          0.296154         0.860787        0.008797         0.057620  
4          0.598784         0.168570        0.686043         0.591674  

Modelo treinado para fazer previsões dos encargos futuros:
[103907.06, 21459.04, 21459.04, 103907.06, 10